## Package imports, data load, and verification

In [1]:
import pandas as pd
import numpy as np

In [2]:
initial = (pd.read_csv("base_stats.csv")
             .rename({'nomr_combined':'norm_combined'},
                     axis=1))

In [3]:
initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   team         24 non-null     int64  
 1   name         24 non-null     object 
 2   dname        24 non-null     object 
 3   kd           24 non-null     float64
 4   norm_kd      24 non-null     float64
 5   kda          24 non-null     float64
 6   most killed  24 non-null     float64
 7   log_norm_mk  24 non-null     float64
 8   win_perc     24 non-null     float64
dtypes: float64(6), int64(1), object(2)
memory usage: 1.8+ KB


In [4]:
initial.head()

,team,name,dname,kd,norm_kd,kda,most killed,log_norm_mk,win_perc
0,1,iron,IRONKNIGHT INC,1.08,0.67500,1.42,35.0,0.617705,0.4427
1,1,mrspikey,AussieCommander,0.83,0.51875,1.15,29.0,0.585032,0.5399
2,1,suffa,SuFFa,0.63,0.39375,0.93,30.0,0.590922,0.3934
3,2,saxonj23,Saxonj23,1.15,0.71875,1.51,35.0,0.617705,0.5292
4,2,dluith,dluith,0.90,0.56250,1.23,32.0,0.602135,0.4802


## Feature Generation and Verification

In [5]:
initial['assists'] = initial.kda-initial.kd
initial['norm_assists'] = initial.assists/initial.assists.max()
initial['combined'] = (initial.norm_kd+initial.log_norm_mk+initial.win_perc+initial.norm_assists)/4
initial['norm_combined'] = initial.combined/initial.combined.max()

In [6]:
initial.head()

,team,name,dname,kd,norm_kd,kda,most killed,log_norm_mk,win_perc,assists,norm_assists,combined,norm_combined
0,1,iron,IRONKNIGHT INC,1.08,0.67500,1.42,35.0,0.617705,0.4427,0.34,0.739130,0.618634,0.765427
1,1,mrspikey,AussieCommander,0.83,0.51875,1.15,29.0,0.585032,0.5399,0.32,0.695652,0.584834,0.723606
2,1,suffa,SuFFa,0.63,0.39375,0.93,30.0,0.590922,0.3934,0.30,0.652174,0.507562,0.627999
3,2,saxonj23,Saxonj23,1.15,0.71875,1.51,35.0,0.617705,0.5292,0.36,0.782609,0.662066,0.819165
4,2,dluith,dluith,0.90,0.56250,1.23,32.0,0.602135,0.4802,0.33,0.717391,0.590557,0.730687


In [7]:
"""
Ratings/elo mapping based on normalized combined score averages

Based on FIDE https://en.wikipedia.org/wiki/Elo_rating_system#Performance_rating
"""

bins = [1.0,.99,.9,.8,.7,.6,.5,.4,.3,.2,.01,0.0,np.NINF]
ratings = [800,677,366,240,149,72,0,-72,-142,-240,-366,-677]
bins.sort()
ratings.sort()
initial['elo'] = pd.cut(initial.norm_combined.values,
                        bins=bins,
                        labels=ratings
                  ).astype(int)

In [8]:
"""
K Factor for updating ELO 
* Based on where you fall within the normalized combined ratings
* 4 different k-values for within the quantiles
* top 25% : 10, 75-50% : 20, 50-25% : 30, 25% below : 40

This uses a modified k-value from FIDE https://en.wikipedia.org/wiki/Elo_rating_system#Most_accurate_K-factor

The idea is:
* 'veteran' players should see less change with wins/loses
* 'novice' players should see more change with wins/loses
"""

quantiles = initial.norm_combined.quantile([0.0,.25,.5,.55,1.0]).values
quantiles[0]=np.NINF
k_value = [5,10,15,20]
initial['k'] = pd.cut(initial.norm_combined.values,
                      bins=quantiles,
                      labels=k_value
                ).astype(int)

In [9]:
"""
Add the team elo calculation back into the initial data set and view
"""
df = (initial.merge(initial.groupby('team')
                           .agg({"elo":'sum'})
                           .reset_index()
                           .rename({'elo':'team_elo'},
                                   axis=1),
                    on='team', 
                    how='left'))
df.head(24)

,team,name,dname,kd,norm_kd,kda,most killed,log_norm_mk,win_perc,assists,norm_assists,combined,norm_combined,elo,k,team_elo
0,1,iron,IRONKNIGHT INC,1.08,0.67500,1.42,35.00,0.617705,0.4427,0.34,0.739130,0.618634,0.765427,240,10,629
1,1,mrspikey,AussieCommander,0.83,0.51875,1.15,29.00,0.585032,0.5399,0.32,0.695652,0.584834,0.723606,240,10,629
2,1,suffa,SuFFa,0.63,0.39375,0.93,30.00,0.590922,0.3934,0.30,0.652174,0.507562,0.627999,149,5,629
3,2,saxonj23,Saxonj23,1.15,0.71875,1.51,35.00,0.617705,0.5292,0.36,0.782609,0.662066,0.819165,366,20,755
4,2,dluith,dluith,0.90,0.56250,1.23,32.00,0.602135,0.4802,0.33,0.717391,0.590557,0.730687,240,10,755
5,2,kry lxxvi,Kry LXXXVI,0.63,0.39375,0.99,21.00,0.528954,0.4686,0.36,0.782609,0.543478,0.672438,149,5,755
6,3,capt_TT,Capt_TT,1.25,0.78125,1.67,43.00,0.653469,0.5395,0.42,0.913043,0.721816,0.893092,366,20,846
7,3,divi,Divi8882,0.81,0.50625,1.14,34.00,0.612668,0.4610,0.33,0.717391,0.574327,0.710607,240,10,846
8,3,nephew of jimi,taybon,0.76,0.47500,1.13,25.00,0.559246,0.4392,0.37,0.804348,0.569448,0.704570,240,5,846
9,4,haydos,Haydos2207,1.23,0.76875,1.59,42.00,0.649381,0.5236,0.36,0.782609,0.681085,0.842697,366,20,755


### Write to .csv for full stats

In [10]:
df.to_csv("full_stats.csv",index=False)

In [11]:
def team_elo_update(df:pd.DataFrame):
    """
    Takes a pandas DataFrame and returns a new DataFrame with just team numbers and team elo
    """
    return (df.groupby('team')
              .agg({"elo":'sum'})
              .reset_index()
              .rename({'elo':'team_elo'},
                      axis=1))

In [12]:
team_elo = team_elo_update(df)
team_elo.head()

,team,team_elo
0,1,629
1,2,755
2,3,846
3,4,755
4,5,846


In [13]:
def prob(teamA:int, teamB:int):
    ea = (1/(1+10**(
        (team_elo[team_elo.team==teamB].team_elo.values[0] -
         team_elo[team_elo.team==teamA].team_elo.values[0])/400))
         ).round(3)
    eb = (1/(1+10**(
        (team_elo[team_elo.team==teamA].team_elo.values[0] -
         team_elo[team_elo.team==teamB].team_elo.values[0])/400))
         ).round(3)
    return(ea,eb)

In [14]:
def prob_calc(teamA:int, teamB:int):
    ea = (1/(1+10**(
        (team_elo[team_elo.team==teamB].team_elo.values[0] -
         team_elo[team_elo.team==teamA].team_elo.values[0])/400))
         ).round(3)
    eb = (1/(1+10**(
        (team_elo[team_elo.team==teamA].team_elo.values[0] -
         team_elo[team_elo.team==teamB].team_elo.values[0])/400))
         ).round(3)
    print("Team",teamA,"has a:",ea,"% chance and Team",teamB,"has a:",eb,"% chance of winning")

## Round 1-7 Initial
### This is done for all 7 initial rounds to show what the probabilities would be without any update to elo
### Subsequent updates for each round will include a reference to these to show the changes

In [15]:
print("Round 1")
# Team 7 v Team 5
prob_calc(7,5)
# Team 4 v Team 2
prob_calc(4,2)
# Team 1 v Team 3
prob_calc(1,3)
# Team 8 v Team 6
prob_calc(8,6)

Round 1
Team 7 has a: 0.857 % chance and Team 5 has a: 0.143 % chance of winning
Team 4 has a: 0.5 % chance and Team 2 has a: 0.5 % chance of winning
Team 1 has a: 0.223 % chance and Team 3 has a: 0.777 % chance of winning
Team 8 has a: 0.67 % chance and Team 6 has a: 0.33 % chance of winning


In [16]:
print("Round 2")
# Team 3 v Team 4
prob_calc(3,4)
# Team 2 v Team 8
prob_calc(2,8)
# Team 5 v Team 1
prob_calc(5,7)
# Team 6 v Team 7
prob_calc(6,7)

Round 2
Team 3 has a: 0.628 % chance and Team 4 has a: 0.372 % chance of winning
Team 2 has a: 0.038 % chance and Team 8 has a: 0.962 % chance of winning
Team 5 has a: 0.143 % chance and Team 7 has a: 0.857 % chance of winning
Team 6 has a: 0.55 % chance and Team 7 has a: 0.45 % chance of winning


In [17]:
print("Round 3")
# Team 4 v Team 5
prob_calc(4,5)
# Team 1 v Team 6
prob_calc(1,6)
# Team 8 v Team 7
prob_calc(8,7)
# Team 2 v Team 3
prob_calc(2,3)

Round 3
Team 4 has a: 0.372 % chance and Team 5 has a: 0.628 % chance of winning
Team 1 has a: 0.038 % chance and Team 6 has a: 0.962 % chance of winning
Team 8 has a: 0.713 % chance and Team 7 has a: 0.287 % chance of winning
Team 2 has a: 0.372 % chance and Team 3 has a: 0.628 % chance of winning


In [18]:
# Round 4
# Team 7 v Team 1
prob_calc(7,1)
# Team 5 v Team 2
prob_calc(5,2)
# Team 6 v Team 4
prob_calc(6,4)
# Team 3 v Team 8
prob_calc(3,8)

Team 7 has a: 0.954 % chance and Team 1 has a: 0.046 % chance of winning
Team 5 has a: 0.628 % chance and Team 2 has a: 0.372 % chance of winning
Team 6 has a: 0.925 % chance and Team 4 has a: 0.075 % chance of winning
Team 3 has a: 0.063 % chance and Team 8 has a: 0.937 % chance of winning


In [19]:
print("Round 5")
# Team 3 v Team 5
prob_calc(3,5)
# Team 4 v Team 7
prob_calc(4,7)
# Team 2 v Team 6
prob_calc(2,6)
# Team 8 v Team 1
prob_calc(8,1)

Round 5
Team 3 has a: 0.5 % chance and Team 5 has a: 0.5 % chance of winning
Team 4 has a: 0.09 % chance and Team 7 has a: 0.91 % chance of winning
Team 2 has a: 0.075 % chance and Team 6 has a: 0.925 % chance of winning
Team 8 has a: 0.981 % chance and Team 1 has a: 0.019 % chance of winning


In [20]:
print("Round 6")
# Team 7 v Team 2
prob_calc(7,2)
# Team 6 v Team 3
prob_calc(6,3)
# Team 5 v Team 8
prob_calc(5,8)
# Team 1 v Team 4
prob_calc(1,4)

Round 6
Team 7 has a: 0.91 % chance and Team 2 has a: 0.09 % chance of winning
Team 6 has a: 0.88 % chance and Team 3 has a: 0.12 % chance of winning
Team 5 has a: 0.063 % chance and Team 8 has a: 0.937 % chance of winning
Team 1 has a: 0.326 % chance and Team 4 has a: 0.674 % chance of winning


In [21]:
print("Round 7")
# Team 2 v Team 1
prob_calc(2,1)
# Team 8 v Team 4
prob_calc(8,4)
# Team 3 v Team 7
prob_calc(3,7)
# Team 5 v Team 6
prob_calc(5,6)

Round 7
Team 2 has a: 0.674 % chance and Team 1 has a: 0.326 % chance of winning
Team 8 has a: 0.962 % chance and Team 4 has a: 0.038 % chance of winning
Team 3 has a: 0.143 % chance and Team 7 has a: 0.857 % chance of winning
Team 5 has a: 0.12 % chance and Team 6 has a: 0.88 % chance of winning


In [22]:
def elo_update(df:pd.DataFrame,teamA:int,teamB:int,result:int):
    """
    Takes a team number as a winner and provides the new elo scores for that team
    """
    ea,eb = prob(teamA,teamB)
    if teamA == result:
        for member in df[df.team==teamA].index:
            df.iloc[member,13]=df.iloc[member,13]+df.iloc[member,14]*(1-ea)
        for member in df[df.team==teamB].index:
            df.iloc[member,13]=df.iloc[member,13]+df.iloc[member,14]*(0-eb)
    else:
        for member in df[df.team==teamA].index:
            df.iloc[member,13]=df.iloc[member,13]+df.iloc[member,14]*(0-ea)
        for member in df[df.team==teamB].index:
            df.iloc[member,13]=df.iloc[member,13]+df.iloc[member,14]*(1-eb)
    return (df.merge(df.groupby('team')
                       .agg({"elo":'sum'})
                       .reset_index()
                       .rename({'elo':'team_elo'},
                               axis=1),
                     on='team', 
                     how='left'))

## Round 2 Updated

In [23]:
team_elo = team_elo_update(initial)
print("Round 2")
# Team 3 v Team 4
prob_calc(3,4)
# Team 2 v Team 8
prob_calc(2,8)
# Team 2 v Team 6
prob_calc(5,1)
# Team 8 v Team 1
prob_calc(6,7)

Round 2
Team 3 has a: 0.628 % chance and Team 4 has a: 0.372 % chance of winning
Team 2 has a: 0.038 % chance and Team 8 has a: 0.962 % chance of winning
Team 5 has a: 0.777 % chance and Team 1 has a: 0.223 % chance of winning
Team 6 has a: 0.55 % chance and Team 7 has a: 0.45 % chance of winning


In [24]:
round2 = initial.copy()
round2 = elo_update(round2,7,5,7)
round2 = elo_update(round2,4,2,4)
#round2 = elo_update(round2,1,3,x)
#round2 = elo_update(round2,8,6,x)

In [25]:
team_elo = team_elo_update(round2)
print("Round 2")
# Team 3 v Team 4
prob_calc(3,4)
# Team 2 v Team 8
prob_calc(2,8)
# Team 2 v Team 6
prob_calc(5,1)
# Team 8 v Team 1
prob_calc(6,7)

Round 2
Team 3 has a: 0.604 % chance and Team 4 has a: 0.396 % chance of winning
Team 2 has a: 0.035 % chance and Team 8 has a: 0.965 % chance of winning
Team 5 has a: 0.769 % chance and Team 1 has a: 0.231 % chance of winning
Team 6 has a: 0.54 % chance and Team 7 has a: 0.46 % chance of winning


## Round 3 Update

In [26]:
team_elo = team_elo_update(initial)
print("Round 3")
# Team 4 v Team 5
prob_calc(4,5)
# Team 1 v Team 6
prob_calc(1,6)
# Team 8 v Team 7
prob_calc(8,7)
# Team 2 v Team 3
prob_calc(2,3)

Round 3
Team 4 has a: 0.372 % chance and Team 5 has a: 0.628 % chance of winning
Team 1 has a: 0.038 % chance and Team 6 has a: 0.962 % chance of winning
Team 8 has a: 0.713 % chance and Team 7 has a: 0.287 % chance of winning
Team 2 has a: 0.372 % chance and Team 3 has a: 0.628 % chance of winning


In [27]:
team_elo = team_elo_update(round2)
print("Round 3")
# Team 4 v Team 5
prob_calc(4,5)
# Team 1 v Team 6
prob_calc(1,6)
# Team 8 v Team 7
prob_calc(8,7)
# Team 2 v Team 3
prob_calc(2,3)

Round 3
Team 4 has a: 0.407 % chance and Team 5 has a: 0.593 % chance of winning
Team 1 has a: 0.038 % chance and Team 6 has a: 0.962 % chance of winning
Team 8 has a: 0.704 % chance and Team 7 has a: 0.296 % chance of winning
Team 2 has a: 0.349 % chance and Team 3 has a: 0.651 % chance of winning
